## Handle Imports

In [147]:
import numpy as np
import pandas as pd
import heapq as pq
import csv
import statistics as stats

# Upload methods

### Load Countries

In [148]:
def load_countries() -> dict:
    countries = {}

    with open("countries.csv") as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row["Country"]
            countries[key] = {name: float(value) for name, value in row.items() if name != "Country"}

    return countries

### Load Resources

In [149]:
def load_resources() -> dict:
    resources = {}

    with open("resources.csv") as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row["Resource"]
            resources[key] = {name: float(value) if name == "Weight" else value for name, value in row.items() if name != "Resource"}

    return resources

## Upload Countries

In [150]:
countryList = load_countries()

In [151]:
resourceList = load_resources()

In [152]:
print(countryList)

{'Atlantis': {'population': 20.0, 'food': 12.0, 'water': 10.0, 'land': 20.0, 'timber': 10.0, 'housing': 5.0, 'metallic element': 3.0, 'metallic alloy': 9.0, 'electronics': 40.0, 'potential fossil energy': 1.0, 'potential fossil usable': 24.0, 'community buildings': 20.0, 'jobs': 10.0, 'high school education': 2.0, 'college education': 12.0, 'universities': 20.0, 'nobel prizes': 10.0, 'marriages': 2.0, 'children': 10.0, 'renewable energy': 3.0, 'food waste': 20.0, 'water waste': 8.0, 'land waste': 10.0, 'timber waste': 4.0, 'housing waste': 3.0, 'metallicAlloy waste': 12.0, 'electronics waste': 14.0, 'potential fossil energy waste': 10.0, 'renewable energy waste': 10.0}, 'Dinotopoia': {'population': 15.0, 'food': 0.0, 'water': 2.0, 'land': 60.0, 'timber': 0.0, 'housing': 0.0, 'metallic element': 4.0, 'metallic alloy': 10.0, 'electronics': 4.0, 'potential fossil energy': 2.0, 'potential fossil usable': 56.0, 'community buildings': 30.0, 'jobs': 2.0, 'high school education': 4.0, 'college

In [153]:
print(resourceList)

{'R1': {'Weight': 0.0, 'Notes': 'analog to population'}, 'R2': {'Weight': 0.0, 'Notes': 'analog to metalic elements'}, 'R3': {'Weight': 0.0, 'Notes': 'analog to timber'}, 'R4': {'Weight': 0.2, 'Notes': 'analog to metallic alloys'}, 'R5': {'Weight': 0.5, 'Notes': 'analog to electronics'}, 'R6': {'Weight': 0.8, 'Notes': 'analog to housing'}, 'R7': {'Weight': -0.5, 'Notes': 'waste'}, 'R8': {'Weight': -0.8, 'Notes': 'waste'}, 'R9': {'Weight': -0.4, 'Notes': 'waste'}, 'R10': {'Weight': -0.2, 'Notes': 'waste'}}


### Set to Pandas Dataframe

In [154]:
countrydf = pd.DataFrame(countryList).transpose()

In [155]:
resourcedf = pd.DataFrame(resourceList).transpose()

In [156]:
countrydf

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,12.0,10.0,20.0,10.0,5.0,3.0,9.0,40.0,1.0,...,3.0,20.0,8.0,10.0,4.0,3.0,12.0,14.0,10.0,10.0
Dinotopoia,15.0,0.0,2.0,60.0,0.0,0.0,4.0,10.0,4.0,2.0,...,2.0,10.0,2.0,0.0,5.0,9.0,3.0,23.0,23.0,3.0
Erewhon,21.0,2.0,3.0,40.0,0.0,30.0,5.0,26.0,2.0,0.0,...,10.0,2.0,1.0,0.0,6.0,1.0,9.0,34.0,33.0,2.0
King's Landing,80.0,3.0,4.0,30.0,0.0,15.0,1.0,35.0,3.0,0.0,...,3.0,3.0,9.0,3.0,1.0,0.0,8.0,52.0,4.0,3.0
The Vale,43.0,1.0,5.0,10.0,0.0,12.0,10.0,30.0,4.0,0.0,...,2.0,4.0,0.0,2.0,2.0,24.0,1.0,10.0,1.0,1.0


In [157]:
resourcedf

,Weight,Notes
R1,0.0,analog to population
R2,0.0,analog to metalic elements
R3,0.0,analog to timber
R4,0.2,analog to metallic alloys
R5,0.5,analog to electronics
R6,0.8,analog to housing
R7,-0.5,waste
R8,-0.8,waste
R9,-0.4,waste
R10,-0.2,waste


### Population Normalization

In [158]:
def normalize_df(df):
    
    for row in range(len(df)):
        
        values = df.iloc[row]
        popVal = values[0]
        
        for vals in range(1, len(values)):
            values[vals] = values[vals]/popVal
        
    return df

In [178]:
norm_df = normalize_df(countrydf)

In [179]:
norm_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,0.030000,0.025000,0.050000,0.025,0.012500,0.007500,0.022500,0.100000,0.002500,...,0.007500,0.050000,0.020000,0.025000,0.010000,0.007500,0.030000,0.035000,0.025000,0.025000
Dinotopoia,15.0,0.000000,0.008889,0.266667,0.000,0.000000,0.017778,0.044444,0.017778,0.008889,...,0.008889,0.044444,0.008889,0.000000,0.022222,0.040000,0.013333,0.102222,0.102222,0.013333
Erewhon,21.0,0.004535,0.006803,0.090703,0.000,0.068027,0.011338,0.058957,0.004535,0.000000,...,0.022676,0.004535,0.002268,0.000000,0.013605,0.002268,0.020408,0.077098,0.074830,0.004535
King's Landing,80.0,0.000469,0.000625,0.004687,0.000,0.002344,0.000156,0.005469,0.000469,0.000000,...,0.000469,0.000469,0.001406,0.000469,0.000156,0.000000,0.001250,0.008125,0.000625,0.000469
The Vale,43.0,0.000541,0.002704,0.005408,0.000,0.006490,0.005408,0.016225,0.002163,0.000000,...,0.001082,0.002163,0.000000,0.001082,0.001082,0.012980,0.000541,0.005408,0.000541,0.000541


# State Quality Function

### Check Country List

In [180]:
for key in countryList:
    print(key, '->', countryList[key])

Atlantis -> {'population': 20.0, 'food': 12.0, 'water': 10.0, 'land': 20.0, 'timber': 10.0, 'housing': 5.0, 'metallic element': 3.0, 'metallic alloy': 9.0, 'electronics': 40.0, 'potential fossil energy': 1.0, 'potential fossil usable': 24.0, 'community buildings': 20.0, 'jobs': 10.0, 'high school education': 2.0, 'college education': 12.0, 'universities': 20.0, 'nobel prizes': 10.0, 'marriages': 2.0, 'children': 10.0, 'renewable energy': 3.0, 'food waste': 20.0, 'water waste': 8.0, 'land waste': 10.0, 'timber waste': 4.0, 'housing waste': 3.0, 'metallicAlloy waste': 12.0, 'electronics waste': 14.0, 'potential fossil energy waste': 10.0, 'renewable energy waste': 10.0}
Dinotopoia -> {'population': 15.0, 'food': 0.0, 'water': 2.0, 'land': 60.0, 'timber': 0.0, 'housing': 0.0, 'metallic element': 4.0, 'metallic alloy': 10.0, 'electronics': 4.0, 'potential fossil energy': 2.0, 'potential fossil usable': 56.0, 'community buildings': 30.0, 'jobs': 2.0, 'high school education': 4.0, 'college e

### Set Maslow Constants

In [162]:
#different resource levels
levelOneResources = {'food': 10, 'water': 20}
levelTwoResources = {'housing': 20, 'timber': 5, 'metallic alloy': 20, 'electronics': 3, 'potential fossil energy': 30} 
levelThreeResources = {'community buildings': 20, 'jobs': 10, 'high school education': 2, 'college education': 10, 'universities': 10, 'marriages': 20, } 
levelFourResources = {'children': 35, 'renewable energy': 30}
levelFiveResources = {'food waste': 2, 'water waste': 2, 'land waste': 1, 'timber waste': 2, 'nobel prizes': 20}

#list of levels
levList = [levelOneResources, levelTwoResources, levelThreeResources, levelFourResources, levelFiveResources]

In [163]:
print(levList)

[{'food': 10, 'water': 20}, {'housing': 20, 'timber': 5, 'metallic alloy': 20, 'electronics': 3, 'potential fossil energy': 30}, {'community buildings': 20, 'jobs': 10, 'high school education': 2, 'college education': 10, 'universities': 10, 'marriages': 20}, {'children': 35, 'renewable energy': 30}, {'food waste': 2, 'water waste': 2, 'land waste': 1, 'timber waste': 2, 'nobel prizes': 20}]


### Level Function

In [164]:
#pandas version
def leveldf(df, country, level, resources):
    
    levelSat = True
    mult = []
    average = 0

    for key, value in levList[level-1].items():
        countryVal = df.loc[country, key]
        mult.append(countryVal/value)
        if countryVal < value:
            levelSat = False
    
    average = stats.mean(mult)
    
    if not levelSat:
        average = average/4
        
    return average

In [165]:
num = leveldf(norm_df, 'Atlantis', 1, levList)
num

0.010624999999999999

In [166]:
#dictionary version (doesn't work)
def level(country, level):

    levelSat = True
    mult = []
    average = 0
    
    #go through each resource of the country
    for key, value in country:
        if key in level.keys():
            mult.append(value/level.keys()[0])
            if value < level.keys()[0]:
                levelSat = False
    
    average = mult.sum()/mult.len()
    
    if not levelSat:
        average = average/4
        
    return average

## Maslow Function

In [167]:
def maslow(df, country, level, resources):
    
    maslowList = []
    
    for num in range(1, level+1):
        levValue = leveldf(countrydf, country, num, levList)
        maslowList.append(levValue)
            
    return maslowList

In [168]:
maslowL = maslow(norm_df, 'The Vale', 5, levList)

In [169]:
print(maslowL)

[0.0010174418604651163, 0.003992248062015504, 0.013953488372093023, 0.008499446290143965, 0.011046511627906977]


In [170]:
def maslowHeuristicVal(lst):
    return stats.mean(maslowL)

In [171]:
masVal = maslowHeuristicVal(maslowL)

In [172]:
print(masVal)

0.007701827242524917


# Transform Templates

In [173]:
HOUSING = [{'land': 1, 'population': 5, 'water': 5, 'metallic element': 1, 'timber': 5, 'metallic alloy': 3, 'potential fossil usable': 5}, {'housing': 1, 'housing waste': 1, 'timber waste': 1, 'population': 5, 'water': 4}]
ALLOYS = [{'population': 1, 'metallic element': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'metallic alloy': 1, 'metallicAlloy waste': 1, 'water': 2}]
ELECTRONICS = [{'population': 1, 'metallic element': 3, 'metallic alloy': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'electronics': 2, 'electronics waste': 1, 'water': 2}]
FARM = [{'population': 1, 'land' : 1, 'water': 3}, {'food': 5, 'population': 1}]
LOGGING = [{'population': 3, 'potential fossil usable': 3}, {'population': 3, 'timber': 5}]
PURIFY_WATER = [{'population': 3, 'potential fossil usable': 3}, {'population': 3, 'water': 5}]
FOSSIL_ENERGY = [{'population': 5, 'potential fossil energy': 2}, {'population': 5, 'potential fossil usable': 1, 'potential fossil energy waste': 1}]
RENEWABLE_ENERGY = [{'population': 5, 'potential fossil usable': 3}, {'population': 5, 'renewable energy': 1, 'renewable energy waste': 1}]
COMMUNITY_BUILDING = [{'land': 1, 'population': 10, 'water': 5, 'metallic element': 3, 'timber': 8, 'metallic alloy': 5, 'potential fossil usable': 5}, {'community buildings': 1, 'housing waste': 1, 'timber waste': 1, 'metallicAlloy waste': 1, 'population': 10, 'water': 4}]
UNIVERSITY = [{'land': 1, 'population': 50, 'water': 5, 'metallic element': 5, 'timber': 10, 'metallic alloy': 5, 'potential fossil usable': 5}, {'universities': 1, 'population': 50, 'water': 3, 'timber waste': 1, 'metallicAlloy waste': 1}]
JOB_HS = [{'population': 25, 'high school education': 1}, {'population': 25, 'jobs': 1}]
JOB_C = [{'population': 50, 'college education': 1}, {'population': 50, 'jobs': 1}]
HIGHSCHOOL_ED = [{'population': 15, 'housing': 1, 'children': 1}, {'population': 16, 'housing': 1, 'high school education': 1}]
COLLEGE_ED = [{'population': 50, 'housing': 1, 'universities': 1, 'high school education': 1}, {'population': 50, 'housing': 1, 'universities': 1, 'college education': 1}]
MARRIAGE = [{'population': 2, 'housing': 1}, {'population': 2, 'housing': 1, 'marriages': 1}]
CHILDREN = [{'marriages': 1, 'housing': 1}, {'marriages': 1, 'housing': 1, 'children': 2}]
NOBEL_PRIZE = [{'population': 1, 'universities': 10, 'college education': 50, 'potential fossil usable': 10}, {'population': 1, 'universities': 10, 'college education': 50, 'nobel prizes': 1}]

# Transform Function

In [174]:
def transform(df, country, transform):
    
    allowed = True
    
    #check if transform is possible
    for key in transform[0]:
        val = transform[0][key]
        if(df.loc[country, key] - val < 0):
            allowed = False
    
    if(allowed):
        #remove input resoures
        for key in transform[0]:
            val = transform[0][key]
            df.loc[country, key] -= val

        #add output resources
        for key in transform[1]:
            val = transform[1][key]
            df.loc[country, key] += val

# Transform Test Case

In [175]:
norm_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,0.600000,0.500000,1.000000,0.5,0.250000,0.150000,0.450000,2.000000,0.050000,...,0.150000,1.000000,0.400000,0.500000,0.200000,0.150000,0.600000,0.700000,0.500000,0.500000
Dinotopoia,15.0,0.000000,0.133333,4.000000,0.0,0.000000,0.266667,0.666667,0.266667,0.133333,...,0.133333,0.666667,0.133333,0.000000,0.333333,0.600000,0.200000,1.533333,1.533333,0.200000
Erewhon,21.0,0.095238,0.142857,1.904762,0.0,1.428571,0.238095,1.238095,0.095238,0.000000,...,0.476190,0.095238,0.047619,0.000000,0.285714,0.047619,0.428571,1.619048,1.571429,0.095238
King's Landing,80.0,0.037500,0.050000,0.375000,0.0,0.187500,0.012500,0.437500,0.037500,0.000000,...,0.037500,0.037500,0.112500,0.037500,0.012500,0.000000,0.100000,0.650000,0.050000,0.037500
The Vale,43.0,0.023256,0.116279,0.232558,0.0,0.279070,0.232558,0.697674,0.093023,0.000000,...,0.046512,0.093023,0.000000,0.046512,0.046512,0.558140,0.023256,0.232558,0.023256,0.023256


In [176]:
transform(countrydf, 'Atlantis', FARM)

In [177]:
countrydf

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,0.600000,0.500000,1.000000,0.5,0.250000,0.150000,0.450000,2.000000,0.050000,...,0.150000,1.000000,0.400000,0.500000,0.200000,0.150000,0.600000,0.700000,0.500000,0.500000
Dinotopoia,15.0,0.000000,0.133333,4.000000,0.0,0.000000,0.266667,0.666667,0.266667,0.133333,...,0.133333,0.666667,0.133333,0.000000,0.333333,0.600000,0.200000,1.533333,1.533333,0.200000
Erewhon,21.0,0.095238,0.142857,1.904762,0.0,1.428571,0.238095,1.238095,0.095238,0.000000,...,0.476190,0.095238,0.047619,0.000000,0.285714,0.047619,0.428571,1.619048,1.571429,0.095238
King's Landing,80.0,0.037500,0.050000,0.375000,0.0,0.187500,0.012500,0.437500,0.037500,0.000000,...,0.037500,0.037500,0.112500,0.037500,0.012500,0.000000,0.100000,0.650000,0.050000,0.037500
The Vale,43.0,0.023256,0.116279,0.232558,0.0,0.279070,0.232558,0.697674,0.093023,0.000000,...,0.046512,0.093023,0.000000,0.046512,0.046512,0.558140,0.023256,0.232558,0.023256,0.023256


# Transfer Function Definition

In [ ]:
def transfer(df, country1, country2, transfer):

    allowed = True

    #check for validityy

    for key in transfer:
        val = transfer[key]
        if (df.loc[country1,key] - val < 0):
            allowed = False

    if(allowed):
        #remove resource from country 1
        for key in transfer:
            val = transfer[key]
            df.loc[country1,key] -= val

        #add resource to country 2
        for key in transfer:
            val = transfer[key]
            df.loc[country2,key] += val



# Transfer Templates

In [ ]:
FOOD = {"food":5}
WATER = {"water":5}
TIMBER = {"timber":5}
METALLIC_ELEMENT = {"metallic element":5}
METALLIC_ALLOY = {"metallic alloy":5}
ELECTRONICS_TRANSFER = {"electronics":5}
POTENTIAL_FOSSIL_ENERGY = {"potential fossil energy":5}
POTENTIAL_FOSSIL_USABLE = {"potential fossil usable":5}
RENEWABLE_ENERGY_TRANSFER = {"renewable energy":5}

# Trade Function Definition

In [ ]:
def trade(df, country1, country2, transfer1, transfer2):
    # A trade deal between two countries country1 and country2
    # for which country uses transfer1 and country2 uses transfer2
    transfer(df, country1, country2, transfer1)
    transfer(df, country2, country1, transfer2)

# Node Definition

In [1]:
class Node:
    def __init__(self, squal, exutil, sched, countrydf):
        self.squal = squal
        self.exutil = exutil
        self.sched = sched
        self.countrydf = countrydf
    def getSqual(self):
        return self.squal
    def getExutil(self):
        return self.exutil
    def getSched(self):
        return self.sched
    def getCountrydf(self):
        return self.countrydf
        

# Bounded Priority Queue Definition

In [2]:
class BoundedPriorityQueue:
    def __init__(self, bound=None):
        self.bound = bound
        self.queue = list()
    def push(self, priority, item):
        if len(self.queue) == self.bound:
            pq.heappushpop(self.queue, (priority, item))
        else:
            pq.heappush(self.queue, (priority, item))
    def pop(self):
        return pq.heapop(self.queue)[-1]
    def clear(self):
        self.queue.clear()
